In [4]:
# === CapBot v1b – Confidence Accuracy Analysis ===
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from pathlib import Path
from datetime import datetime

# === Setup ===
today = datetime.today().strftime("%Y%m%d")
version = "v1b"
notebook_dir = Path(__file__).parent if "__file__" in globals() else Path().resolve()
version_dir = notebook_dir
report_path = version_dir / f"CapBot_{version}_Confidence_Report_{today}.csv"
readme_path = version_dir / f"CapBot_{version}_README.md"
xlsx_path = version_dir / f"CapBot_{version}_Confidence_Report_{today}.xlsx"

# === Load Data ===
file_path = version_dir / "../../../data/historical/processed/matches_2015_2025_combined_balanced.csv"
df = pd.read_csv(file_path)
numeric_cols = ['rank_A', 'rank_B', 'pts_A', 'pts_B', 'odds_A', 'odds_B']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
df = df.dropna(subset=numeric_cols + ['winner_code'])

# === Train on Full Dataset ===
X = df[numeric_cols]
y = df['winner_code']
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

# === Predict on Full Dataset ===
preds_proba = model.predict_proba(X)[:, 1]
preds = (preds_proba > 0.5).astype(int)
df['pred_proba'] = preds_proba
df['predicted'] = preds
df['correct'] = (preds == y).astype(int)

# === Analyze Confidence Thresholds ===
thresholds = [0.6, 0.7, 0.8, 0.9]
rows = []

for threshold in thresholds:
    confident = df[df['pred_proba'] >= threshold]
    total = len(confident)
    correct = confident['correct'].sum()
    accuracy = correct / total if total > 0 else 0
    rows.append({
        "Threshold": f">= {threshold}",
        "Total Predictions": total,
        "Correct Predictions": correct,
        "Accuracy": round(accuracy, 4)
    })

# === Output Results ===
summary_df = pd.DataFrame(rows)
summary_df.to_csv(report_path, index=False)

# Save to XLSX with total match summary
with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
    summary_df.to_excel(writer, sheet_name="Confidence Summary", index=False)
    workbook  = writer.book
    worksheet = writer.sheets["Confidence Summary"]
    worksheet.write(len(summary_df) + 2, 0, "Total Matches")
    worksheet.write(len(summary_df) + 2, 1, len(df))
    for i, row in enumerate(rows):
        threshold_val = float(row["Threshold"].split()[-1])
        count = row["Total Predictions"]
        acc = row["Accuracy"]
        worksheet.write(len(summary_df) + 3 + i, 0, f"≥ {threshold_val}")
        worksheet.write(len(summary_df) + 3 + i, 1, f"{count} matches")
        worksheet.write(len(summary_df) + 3 + i, 2, f"{acc:.2%} accuracy")

print(summary_df)
print(f"\n📄 Report saved to: {report_path.resolve()}")
print(f"📊 XLSX saved to: {xlsx_path.resolve()}")

# === Generate README ===
total_matches = len(df)
summary_lines = [f"Out of **{total_matches} total matches**:"]

for threshold in [0.5, 0.6, 0.7, 0.8, 0.9]:
    confident = df[df['pred_proba'] >= threshold]
    total = len(confident)
    correct = confident['correct'].sum()
    accuracy = correct / total if total > 0 else 0
    summary_lines.append(f"- **{total} matches** had `pred_proba ≥ {threshold}` → Accuracy: **{accuracy:.2%}**")

confidence_summary = "\n".join(summary_lines)

readme = f"""\
# 📄 CapBot {version.upper()} – Confidence Calibration Report

**Date:** {today}  
**Report File:** `{report_path.name}`
**Excel Report:** `{xlsx_path.name}`

---

## 🎯 Objective
To evaluate the baseline model's confidence calibration by analyzing accuracy at various prediction probability thresholds using the **entire historical dataset** (~{total_matches} matches).

---

## 🧠 Methodology
- Trained a logistic regression model using all historical matches
- Predicted win probabilities (`pred_proba`) for each match
- Measured how accurate the model is when its confidence exceeds specific thresholds

---

## 📊 Confidence Threshold Summary
{confidence_summary}

---

## 📈 Raw Table View
{summary_df.to_markdown(index=False)}

---

## ✅ Insights
- Confidence thresholds correlate positively with accuracy
- Higher confidence yields fewer predictions, but better reliability
- This lays the groundwork for smarter filtering and EV-based strategies

---
"""

with open(readme_path, "w", encoding="utf-8") as f:
    f.write(readme)

print(f"📄 README saved to: {readme_path.resolve()}")


/var/folders/1r/xbh8qf_s1cb25q4t_1_v4bzh0000gn/T/ipykernel_88979/4211160408.py:20: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


  Threshold  Total Predictions  Correct Predictions  Accuracy
0    >= 0.6              15260                11565    0.7579
1    >= 0.7               8345                 6898    0.8266
2    >= 0.8               4140                 3658    0.8836
3    >= 0.9               1930                 1792    0.9285

📄 Report saved to: /Users/boroni_4/Documents/CapBot/capbot/notebooks/versions/v1b/CapBot_v1b_Confidence_Report_20250516.csv
📊 XLSX saved to: /Users/boroni_4/Documents/CapBot/capbot/notebooks/versions/v1b/CapBot_v1b_Confidence_Report_20250516.xlsx
📄 README saved to: /Users/boroni_4/Documents/CapBot/capbot/notebooks/versions/v1b/CapBot_v1b_README.md
